# Compute Functions and Jobs
__________________
The Compute module provides scalable compute capabilities to parallelize your analysis. Compute enables users to package and execute your Python code within nodes hosted on Descartes Lab's cloud infrastructure. These nodes offer the ability to access imagery at extremely high rates of throughput to execute computations over nearly any spatio-temporal scale. 

This example provides a light introduction to the basics of asynchronous computing with Descartes Labs. For a more detailed look at all its classes and their available methods please visit the [`Compute`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html) documentation page.

We'll start by importing the two primary objects within the API, as well as their associated status objects:

 * [`Function:`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) dynamically created, serverless functions containing user specified, compiled code that you can submit many jobs to.
 * [`Job:`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job) submitted request for a single invocation of a created function. 

In [ ]:
from descarteslabs.auth import Auth
from descarteslabs.catalog import Blob, properties as p, StorageType
from descarteslabs.compute import Function, FunctionStatus, Job, JobStatus

We'll also need these imports for this example.

In [ ]:
import sys
from datetime import datetime

## Hello World
Next, we'll create a very basic *hello_world* function which returns a string constructed from the passed argument:

In [ ]:
def hello(arg):
    print(f"Hello, {arg}")
    return f"hello {arg}"

### Creating an Asynchronous Compute Function
To create the object, we simply need to call [`Function()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) and specify the desired parameters. 

The recommended minimum parameters are your Python function, a name, and the image that will be used to build the Function environment.  Some common attributes used to customize the performance of your scalable compute object include: 
 * __image__ = Reference to the base Compute image to use, which must match your local Python version
 * __cpus__ = number of CPUs requested for a single job
 * __memory__ = max memory available for each job
 * __maximum_concurrency__ = max number of jobs to run in parallel
 * __timeout__ = max length a job can run in seconds
 * __retry_count__ = max number of times a job can be retried
 * __requirements__ = list of Python dependencies required by this function

Once we have defined the object, we simply call [`Function.save()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.save) to complete the creation. 


*__Note__*: For all available **cpus** and **memory** options please visit the [Compute limits](https://docs.descarteslabs.com/guides/quota.html#general-limitations) page.

In [ ]:
major = sys.version_info.major
minor = sys.version_info.minor
image = f"python{major}.{minor}:latest"
image

In [ ]:
print("Creating function")
async_func = Function(
    hello,
    name="my-compute-hello",
    image=image,
    cpus=0.25,
    memory=512,
    maximum_concurrency=20,
    timeout=300,
    retry_count=0,
)
async_func.save()
print(f"Created: {async_func.id}")

*__Note__*: This function will take just a few minutes to build!

## Submitting Jobs
Now that we have a function, we can test it by creating and submitting a job. There are several ways you can submit jobs to a function:
 * `async_func(args)` - Pass arguments directly to the Function
 * [`async_func.map()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.map) - Submit multiple jobs efficiently (discussed in more detail within the next tutorial [02 Create Imagery.ipynb](./02%20Create%20Imagery.ipynb))
 
Below we will invoke our asynchronous function by passing the string *"Hello from my function!"*:

In [ ]:
# invoke the function
print("Submitting a job")
job = async_func("Hello from my function!")
print(f"Submitted: {job.id}")

## Waiting for Completion

Now that we have submitted a job, there are a few ways to wait for completion. It is highly recommended to navigate to the Compute monitor app at [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) anyways to track your progress. 

1. Wait on an individual job via [`Job.wait_for_completion()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job.wait_for_completion)

In [ ]:
job.wait_for_completion()
job.status

2. Wait for all pending and running jobs for the function to complete using [`Function.wait_for_completion()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.wait_for_completion)

In [ ]:
async_func.wait_for_completion()
async_func.status

3. Navigate to the Compute monitor app at [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute)

### Note - to continue this tutorial you must wait for your Function to complete!

## Logging and Results - Functions
When a function build is completed you can view the build logs by calling [`Function.build_log()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.build_log):

In [ ]:
async_func.build_log()

## Logging and Results - Jobs
When a job is completed you can access both the logs and results by [`Job.log()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job.log) and [`Job.result()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job.result) respectively:

In [ ]:
job.log()

#### Note on Job Logs 
These are where you can debug your code if it is failing!

In [ ]:
job.result()

## Logging and Results - Retrieving Results in Bulk

In addition to accessing your build logs and results as shown above, you can also
access these as [`Blob`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob)s, as all these artifacts are automatically stored in the Catalog. 

*__Note__*: Logs are retained only for a period of 30 days. Results are retained indefinitely, even after the Function
and its Jobs have been deleted.

In order to access these artifacts, you will need to form the correct IDs to retrieve them. All three require
the correct [`StorageType`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job) value (__logs__ for logs and __compute__ for results), namespace (available as
[`Function.namespace`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.namespace) while your function exists, or can be derived from your authentication details as shown below),
and the name of the blob derived from your __Function.id__ and __Job.id__.


__Function Build Log id:__

    logs/{your-default-namespace}/{function-id}

    
__Job Execution Log id:__

    logs/{your-default-namespace}/{function-id}/{job-id}

    
__Job Result id:__
        
    compute/{your-default-namespace}/{function-id}/{job-id}

In [ ]:
auth = Auth.get_default_auth()
namespace = (auth.payload.get("org", "") + ":" + auth.namespace).lstrip(":")
print(f"Namespace: {namespace}")

In [ ]:
print(f"Results for {async_func.id}")
for b in (
    Blob.search()
    .filter(p.namespace == namespace)
    .filter(p.name.startswith(f"{async_func.id}/"))
    .filter(p.storage_type == StorageType.COMPUTE)
):
    print(f"ID: {b.id}")
    print(b.data())
    print("\n")

print(f"Job Execution Logs for {async_func.id}")
for b in (
    Blob.search()
    .filter(p.namespace == namespace)
    .filter(p.name.startswith(f"{async_func.id}/"))
    .filter(p.storage_type == StorageType.LOGS)
):
    print(f"ID: {b.id}")
    print(b.data())
    print("\n")

_Note that the build log content is compressed!_

## Searching Functions
You can also search, filter, and sort your previously created functions. Here we will find all of our functions created today:

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")

for func in Function.search().filter(Function.creation_date > today):
    print(func.id)
    print(func.creation_date)
    print(func.status)
    print(len(list(func.jobs)))

We can get active functions by specifying the **ready** status

In [ ]:
active_funcs = [
    {"name": f.name, "id": f.id}
    for f in Function.search().filter(Function.status == FunctionStatus.READY)
]
active_funcs

Here we can filter by prefix:

In [ ]:
my_funcs = [f.name for f in Function.search().filter(Function.name.startswith("my"))]
my_funcs

## Deleting Functions

In order to release all resources associated with a function you should delete it when you are done by calling [`Function.delete()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.delete).

*__Notes on Deletion__*: 
*  Any jobs must have been completed before you can delete the function. When deleted, all associated jobs, build logs, and job logs will be deleted. 
* Results will not be deleted and will remain available unless you call [`Function.delete_jobs(delete_results=True)`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.delete_jobs)

In [ ]:
async_func.delete_jobs(delete_results=True)
async_func.delete()